# `卡方檢定 - Python實戰：商務資料結構整理`
作者：徐子皓
***

## 基礎資料整理

### 讀取資料集，挑選出系列1的資料

In [1]:
import pandas as pd
data = pd.read_csv('電商交易資料.csv', encoding = 'utf-8-sig')
data = data[data['系列']=='系列1']
data.head()

,顏色,單價,成本,系列,產品,訂單時間,會員,性別,年紀,廣告代號all,尺寸,訂單編號
74181,whitetrigray,613.585,450.730,系列1,產品1-11,2016-01-02T14:16:32,L_CBY_00946,FEMALE,32.0,廣告_critei_critei,L,187164
74182,trigrayblack,651.420,481.985,系列1,產品1-11,2016-01-02T14:16:32,L_CBY_00946,FEMALE,32.0,廣告_critei_critei,L,240855
74183,watermelonred,713.930,416.185,系列1,產品1-12,2016-01-03T12:25:18,B_GSJ_67478,FEMALE,26.0,廣告_自然流量,M,241008
74184,jeanblue,768.215,452.375,系列1,產品1-12,2016-01-03T12:25:18,B_GSJ_67478,FEMALE,26.0,廣告_自然流量,M,400758
74185,navyblue,1138.340,697.480,系列1,產品1-13,2016-01-03T12:25:18,B_GSJ_67478,FEMALE,26.0,廣告_自然流量,L,457533


### 挑選出廣告系列名稱

In [2]:
data['廣告代號'] = data['廣告代號all'].str.split('_', expand=True)[1].tolist()
data['廣告代號'][:10]

74181    critei
74182    critei
74183      自然流量
74184      自然流量
74185      自然流量
74186      自然流量
74187      自然流量
74188      自然流量
74189      自然流量
74190      自然流量
Name: 廣告代號, dtype: object

### 保留所需欄位，刪除遺失值

In [3]:
data = data[['廣告代號', '性別']] # 保留所需欄位
data = data.dropna() # 刪除遺失值
data

,廣告代號,性別
74181,critei,FEMALE
74182,critei,FEMALE
74183,自然流量,FEMALE
74184,自然流量,FEMALE
74185,自然流量,FEMALE
...,...,...
103313,自然流量,MALE
103314,自然流量,MALE
103315,自然流量,MALE
103316,自然流量,MALE


In [4]:
data = data.reset_index(drop=True)
data

,廣告代號,性別
0,critei,FEMALE
1,critei,FEMALE
2,自然流量,FEMALE
3,自然流量,FEMALE
4,自然流量,FEMALE
...,...,...
26484,自然流量,MALE
26485,自然流量,MALE
26486,自然流量,MALE
26487,自然流量,MALE


## 進階資料整理

### 計算廣告頻率

In [5]:
from collections import Counter
frequency_count = Counter(data['廣告代號'])

### 製作廣告頻率表

In [6]:
key = list(frequency_count.keys()) # 取出廣告名稱
value = list(frequency_count.values()) # 取出廣告頻率
frequency_table = pd.DataFrame(zip(key,value), columns=['廣告名稱','頻率'])
frequency_table.head()

,廣告名稱,頻率
0,critei,2145
1,自然流量,11430
2,B2K,3
3,ikip,735
4,qdwit,103


### 根據廣告頻率，由大至小排序

In [7]:
frequency_table = frequency_table.sort_values(by = '頻率', ascending = False)
frequency_table.head()

,廣告名稱,頻率
1,自然流量,11430
11,KDP,4118
0,critei,2145
6,KDPOD,1660
24,B2KMP,864


### 取出前三主力廣告

In [8]:
frequency_table = frequency_table[frequency_table['廣告名稱']!='自然流量']
ad_name = frequency_table['廣告名稱'].tolist()[:3]
print('前三主力廣告為：', ad_name)

前三主力廣告為： ['KDP', 'critei', 'KDPOD']


### 保留內含主力廣告之資料

In [9]:
data = data[data['廣告代號'].isin(ad_name)]
data.head()

,廣告代號,性別
0,critei,FEMALE
1,critei,FEMALE
40,critei,MALE
56,critei,FEMALE
57,critei,FEMALE


### 計算不同廣告、性別組合下的資料數量

In [10]:
male_list = []
female_list = []
# 透過迴圈，依序使用不同主力廣告
for ad in ad_name:
    # 男性
    male_count = len(data[data['性別']=='MALE'][data['廣告代號']==ad])
    male_list.append(male_count)
    print('男性，','且受廣告'+ad+'促成的交易共有', male_count, '筆')
    # 女性
    female_count = len(data[data['性別']=='FEMALE'][data['廣告代號']==ad])
    female_list.append(female_count)
    print('女性，','且受廣告'+ad+'促成的交易共有', female_count, '筆')

男性， 且受廣告KDP促成的交易共有 2038 筆
女性， 且受廣告KDP促成的交易共有 2080 筆
男性， 且受廣告critei促成的交易共有 1106 筆
女性， 且受廣告critei促成的交易共有 1039 筆
男性， 且受廣告KDPOD促成的交易共有 743 筆
女性， 且受廣告KDPOD促成的交易共有 917 筆
C:\Users\user\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\user\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


### 製作性別－廣告交叉頻率表

In [11]:
chi_table = pd.DataFrame(zip(male_list, female_list), columns=['男性', '女性'],index=ad_name)
chi_table

,男性,女性
KDP,2038,2080
critei,1106,1039
KDPOD,743,917


***

# `卡方檢定 - Python實戰：掌握不同系列產品推出的族群比重`
作者：徐子皓
***

## 卡方檢定

### 引入相關套件包

In [12]:
import numpy as np
import scipy.stats
from scipy.stats import chisquare
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import plotly.offline as py

### 進行卡方檢定

In [13]:
scipy.stats.chi2_contingency(chi_table, correction = False)

(17.963949463715462,
 0.0001256544684594472,
 2,
 array([[2020.27842989, 2097.72157011],
        [1052.33055661, 1092.66944339],
        [ 814.3910135 ,  845.6089865 ]]))

### 取出卡方檢定p值

In [14]:
p_value = scipy.stats.chi2_contingency(chi_table, correction = False)[1]
p_value

0.0001256544684594472

## 事後檢定

In [15]:
for ind in chi_table.index:
    # 挑選出該廣告促成的男女銷售量
    ad_data = chi_table.loc[ind].tolist()
    # 進行事後檢定
    print('廣告系列'+ind+'的p值為:', chisquare(ad_data)[1])

廣告系列KDP的p值為: 0.5127931549822997
廣告系列critei的p值為: 0.1479971375022954
廣告系列KDPOD的p值為: 1.9489356101720793e-05


## 視覺化呈現

### 繪圖

In [16]:
fig = go.Figure()
# 男性分布圖
fig.add_trace(go.Scatter(
            x= ad_name,
            y= chi_table["男性"].tolist(),
            mode="lines+markers",
            textfont=dict(
            family="sans serif",
            size=16,
            color="royalblue"),    
            line=dict(color='royalblue', width=2),
            name = '男性'
            ))
# 女性分布圖
fig.add_trace(go.Scatter(
            x= ad_name,
            y= chi_table["女性"].tolist(),
            mode="lines+markers",
            textfont=dict(
            family="sans serif",
            size=16,
            color="firebrick"),    
            line=dict(color='firebrick', width=2),
            name = '女性'
            ))

### 佈景主題設定

In [17]:
fig.update_layout(
    title={
        'text': "<b>廣告－性別交叉比較圖</b>",
        'y':0.95, 
        'x':0.5, 
        'xanchor': 'center',},
    yaxis_title='消費次數',
    xaxis={
        'title': '廣告類別',
        'tickmode': 'linear'},
    width=1800, 
    height=960,
    font=dict(
        family="Courier New, monospace",
        size=20, color="lightslategrey"))

### 將產出另存新檔

In [18]:
# 產出另存html檔
py.plot(fig, filename='CH4-7產出成果_廣告－性別交叉比較圖', auto_open=True)
# 產出另存png檔
fig.write_image("CH4-7產出成果_廣告－性別交叉比較圖.png")

C:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\plotly\offline\offline.py:563: UserWarning:

Your filename `CH4-7產出成果_廣告－性別交叉比較圖` didn't end with .html. Adding .html to the end of your file.



## 製作管理決策表

In [19]:
weight = []
for ind in chi_table.index:
    # 挑選出該廣告促成的男女銷售量
    ad_data = chi_table.loc[ind].tolist()
    # 取得事後檢定的p值
    p = chisquare(ad_data)[1]
    # 判斷廣告對於性別的推廣效果
    if p >= 0.05:
        weight.append('男女推廣效果均勻')
    else:
        male = ad_data[0] # 男性消費量
        female = ad_data[1] # 女性消費量
        if male>female:
            weight.append('針對男性族群推廣較有效')
        else:
            weight.append('針對女性族群推廣較有效')            
# 將結果存至比較表中
chi_table['加強推廣族群'] = weight
chi_table

,男性,女性,加強推廣族群
KDP,2038,2080,男女推廣效果均勻
critei,1106,1039,男女推廣效果均勻
KDPOD,743,917,針對女性族群推廣較有效


### 另存管理決策表

In [20]:
chi_table.to_csv("CH4-7產出成果_廣告－性別交叉比較表.csv", encoding='utf-8-sig')